In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
from torch import nn
import torchvision
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split
import torch.nn.functional as F
import datetime
import pandas as pd
import sys
import os
# new_dir = os.path.join('/'.join(os.path.abspath(os.getcwd()).split('/')[:-1]))
# sys.path.append(new_dir)
# from up_down_model import Up_down_flip_model


# prepare data

In [2]:
with open('config.npy', 'rb') as f:
    config = np.load(f)
    sample_num = config.shape[0]
    config = config.reshape([sample_num,2,4,4])
theta,beta = config[:,0,:,:], config[:,1,:,:]
z = np.cos(theta)
z[z<0] = 0

with open('J.npy', 'rb') as f:
    J = np.load(f)


with open('energy.npy', 'rb') as f:
    # energy 都是负的，在这里取 正值
    energy = -np.load(f)


In [3]:
data = TensorDataset(torch.tensor(J).float(),torch.tensor(z).float())

n_train = int(len(data)*0.8)
n_valid = len(data) - n_train
ds_train,ds_valid = random_split(data,[n_train,n_valid])

dl_train,dl_valid = DataLoader(ds_train,batch_size = 8),DataLoader(ds_valid,batch_size = 8)


In [4]:
for features,labels in dl_train:
    print(features.shape)
    print(labels.shape)
    break

torch.Size([8, 32])
torch.Size([8, 4, 4])


# prepare model

### convert layer

In [5]:
class Expander(nn.Module):
    def __init__(self, in_para=32, out_para=480000):
        super().__init__()
        self.linear = torch.nn.Linear(in_para,out_para)
    def forward(self,x):
        x = self.linear(x)
        sample_num = x.shape[0]
        x = x.reshape(sample_num,3,400,400)
        return x

In [6]:
expander = Expander()
expander(torch.tensor(J).float()).shape

torch.Size([1000, 3, 400, 400])

### Encoder

In [7]:
class Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3)
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3)
    
    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))

class Encoder(nn.Module):
    def __init__(self, chs=(3,64,128,256,512,1024)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    
    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs


In [8]:
encoder = Encoder()
# input image
x    = torch.randn(1, 3, 572, 572)
ftrs = encoder(x)
for ftr in ftrs: print(ftr.shape)

/home/yibo/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([1, 64, 568, 568])
torch.Size([1, 128, 280, 280])
torch.Size([1, 256, 136, 136])
torch.Size([1, 512, 64, 64])
torch.Size([1, 1024, 28, 28])


### Decoder

In [9]:
class Decoder(nn.Module):
    def __init__(self, chs=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.chs        = chs
        self.upconvs    = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)]) 
        
    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x)
            enc_ftrs = self.crop(encoder_features[i], x)
            x        = torch.cat([x, enc_ftrs], dim=1)
            x        = self.dec_blocks[i](x)
        return x
    
    def crop(self, enc_ftrs, x):
        _, _, H, W = x.shape
        enc_ftrs   = torchvision.transforms.CenterCrop([H, W])(enc_ftrs)
        return enc_ftrs


In [10]:
decoder = Decoder()
x = torch.randn(1, 1024, 28, 28)
decoder(x, ftrs[::-1][1:]).shape


torch.Size([1, 64, 388, 388])

## shrinker

In [11]:
class Shrinker(nn.Module):
    def __init__(self,input_H,input_W):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(input_H*input_W,16)
    def forward(self,x):
        sample_num = x.shape[0]
        x = self.flatten(x)
        x = self.linear(x)
        return x
        


In [12]:
shrinker = Shrinker(228,228)
input_data = torch.randn(32,1,228,228)
shrinker(input_data).shape

torch.Size([32, 16])

# Unet

In [13]:
class UNet(nn.Module):
    def __init__(self, enc_chs=(3,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=1, retain_dim=False, out_sz=(572,572)):
        super().__init__()
        self.expander    = Expander()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = nn.Conv2d(dec_chs[-1], num_class, 1)
        self.retain_dim  = retain_dim
        self.out_sz      = out_sz
        self.shrinker    = Shrinker(212,212)

    def forward(self, x):
        x        = self.expander(x)
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        if self.retain_dim:
            out = F.interpolate(out, self.out_sz)
        out      = self.shrinker(out)
        return out


In [15]:
unet = UNet()
test_J = torch.tensor(J[:2,:]).float()
unet(test_J).shape

torch.Size([2, 16])

# training

In [ ]:
# from torchkeras import summary
# summary(model,input_shape= (1000,32))

In [ ]:
model.optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
model.loss_func = torch.nn.MSELoss()
model.metric_func = lambda y_pred,y_true: torch.mean(1-torch.abs(y_true-y_pred))
model.metric_name = "auc"

In [ ]:
def train_step(model,features,labels):
    
    # 训练模式，dropout层发生作用
    model.train()
    
    # 梯度清零
    model.optimizer.zero_grad()
    
    # 正向传播求损失
    predictions = model(features)
    loss = model.loss_func(predictions,labels)
    metric = model.metric_func(predictions,labels)

    # 反向传播求梯度
    loss.backward()
    model.optimizer.step()

    return loss.item(),metric.item()

def valid_step(model,features,labels):
    
    # 预测模式，dropout层不发生作用
    model.eval()
    # 关闭梯度计算
    with torch.no_grad():
        predictions = model(features)
        loss = model.loss_func(predictions,labels)
        metric = model.metric_func(predictions,labels)
    
    return loss.item(), metric.item()


# 测试train_step效果
features,labels = next(iter(dl_train))
labels = labels.reshape(8,1)
train_step(model,features,labels)

In [ ]:
def train_model(model,epochs,dl_train,dl_valid,log_step_freq):

    metric_name = model.metric_name
    print("Start Training...")
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("=========="*8 + "%s"%nowtime)
    df = pd.DataFrame([], columns=['EPOCH','loss','auc','val_loss','val_auc'])
    for epoch in range(1,epochs+1):  

        # 1，训练循环-------------------------------------------------
        loss_sum = 0.0
        metric_sum = 0.0
        step = 1

        for step, (features,labels) in enumerate(dl_train, 1):

            loss,metric = train_step(model,features,labels)

            # 打印batch级别日志
            loss_sum += loss
            metric_sum += metric
            if step%log_step_freq == 0:   
                print(("[step = %d] loss: %.3f, "+metric_name+": %.3f") %
                      (step, loss_sum/step, metric_sum/step))

        # 2，验证循环-------------------------------------------------
        val_loss_sum = 0.0
        val_metric_sum = 0.0
        val_step = 1

        for val_step, (features,labels) in enumerate(dl_valid, 1):

            val_loss,val_metric = valid_step(model,features,labels)

            val_loss_sum += val_loss
            val_metric_sum += val_metric

        # 3，记录日志-------------------------------------------------
        info = (epoch, loss_sum/step, metric_sum/step, 
                val_loss_sum/val_step, val_metric_sum/val_step)
        df = df.append(pd.DataFrame([info], columns=['EPOCH','loss','auc','val_loss','val_auc']),ignore_index=True)
        # 打印epoch级别日志
        print(("\nEPOCH = %d, loss = %.3f,"+ metric_name + \
              "  = %.3f, val_loss = %.3f, "+"val_"+ metric_name+" = %.3f") 
              %info)
        nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print("\n"+"=========="*8 + "%s"%nowtime)

    print('Finished Training...')
    
    return df

In [ ]:
epochs = 10

history = train_model(model,epochs,dl_train,dl_valid,log_step_freq = 50)

In [ ]:
# 测试model效果
features_test,labels_test = next(iter(dl_valid))
# labels = labels.reshape(8,1)
predictions_test = model(features_test)